# RAG Pipeline - Data Ingestion (Context and Knowledge)

Copyright 2025-2026, Denis Rothman


Notebook này giải quyết bước đầu tiên cực kỳ quan trọng trong việc xây dựng một **RAG pipeline** phức tạp: **data ingestion** (nạp dữ liệu). Về cơ bản, chúng ta đang thiết lập "bộ não" cho **agent** bằng cách cung cấp cho nó hai loại thông tin riêng biệt—ngữ cảnh về quy trình (**procedural "how-to" context**) và kiến thức thực tế (**factual "what-is" knowledge**). Khi kết thúc, bạn sẽ có một **Pinecone vector database** đã được chuẩn bị đầy đủ, sẵn sàng vận hành một AI thông minh hơn và có khả năng nhận biết ngữ cảnh (**context-aware**).

Dưới đây là tóm tắt các nội dung mà notebook này thực hiện:

* **Environment Setup (Thiết lập môi trường):** Cài đặt tất cả các thư viện Python cần thiết (`openai`, `pinecone`, `tiktoken`, v.v.) và cấu hình bảo mật cho các **API keys**.
* **Vector Database Prep (Chuẩn bị cơ sở dữ liệu vector):** Kết nối với **Pinecone index**, tạo mới nếu chưa tồn tại và xóa dữ liệu cũ để đảm bảo một hệ thống sạch (**clean slate**).
* **Procedural Context (Ngữ cảnh quy trình):** Định nghĩa và tạo **embeddings** cho các "Semantic Blueprints" — các hướng dẫn về phong cách và cấu trúc của chúng ta — sau đó tải chúng lên một **namespace** chuyên dụng có tên là `ContextLibrary`.
* **Factual Knowledge (Kiến thức thực tế):** Thực hiện **chunking** (chia nhỏ) văn bản thô thành các đoạn tối ưu bằng cách sử dụng một **tokenizer**, tạo **embeddings** cho chúng và tải toàn bộ lên một **namespace** riêng biệt gọi là `KnowledgeStore` để phục vụ việc truy xuất dữ liệu thực tế (**factual recall**).


In [1]:
# Imports for this notebook
import json
import time
from tqdm.auto import tqdm
import tiktoken
from pinecone import Pinecone, ServerlessSpec
from tenacity import retry, stop_after_attempt, wait_random_exponential
# general imports required in the notebooks of this book
import re
import textwrap
from IPython.display import display, Markdown
import copy

# ServerlessSpec: dùng để tạo index vector
# retry: dùng để retry khi có lỗi
# stop_after_attempt: dùng để stop khi retry quá lần
# wait_random_exponential: dùng để wait random exponential, nghĩa là đợi ngẫu nhiên


/home/dino/Documents/learning-2026/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

# The client will automatically read the OPENAI_API_KEY from your environment.
client = OpenAI()
print("OpenAI client initialized.")

OpenAI client initialized.


In [5]:
# Configuration
EMBEDDING_MODEL = "text-embedding-3-small"
EMBEDDING_DIM = 1536 # Dimension for text-embedding-3-small
GENERATION_MODEL = "gpt-5"

In [7]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

## 2.Initialize Clients


### 1. Các khái niệm cơ bản và cách tổ chức dữ liệu trong Pinecone
Pinecone là một **cơ sở dữ liệu vector (vector database)** được sử dụng để lưu trữ và truy xuất các bản biểu diễn số học (embeddings) của dữ liệu. Trong hệ thống này, dữ liệu được tổ chức theo cấu trúc phân cấp:

*   **Index (Chỉ mục):** Đây là đơn vị lưu trữ cao nhất trong Pinecone, giống như một cơ sở dữ liệu riêng biệt. Một chỉ mục sẽ giữ toàn bộ các vector cần thiết cho dự án.
*   **Namespace (Không gian tên):** Đây là một tính năng cho phép **chia một chỉ mục duy nhất thành các phần tách biệt nghiêm ngặt**,. Điều này cực kỳ quan trọng trong kiến trúc **RAG kép (dual RAG)** để quản lý các loại dữ liệu khác nhau mà không bị lẫn lộn.

### 2. Vai trò của Index Name và Namespace Name
Trong bối cảnh của tài liệu, việc phân chia này phục vụ một mục đích chiến lược:

*   **Index Name (Tên chỉ mục):** Định danh duy nhất cho kho lưu trữ vector của dự án (ví dụ: `genai-mas-mcp-ch3`). Nó xác định phạm vi tài nguyên mà hệ thống sẽ truy cập.
*   **Namespace Name (Tên không gian tên):**
    *   **KnowledgeStore:** Dùng để lưu trữ các vector từ **dữ liệu thực tế (factual data)**,. Đây là nơi tác nhân Nhà nghiên cứu (Researcher agent) tìm kiếm thông tin để trả lời câu hỏi "cái gì".
    *   **ContextLibrary:** Dùng để lưu trữ các vector từ **bản thiết kế ngữ nghĩa (semantic blueprints)** hoặc các hướng dẫn quy trình,. Đây là nơi tác nhân Thủ thư (Librarian agent) truy tìm "cách thức" để cấu trúc phản hồi.

### 3. Giải thích chi tiết đoạn code khởi tạo

Đoạn code của bạn thực hiện các bước thiết lập nền tảng sau:

*   **`pc = Pinecone(api_key=PINECONE_API_KEY)`:** Dòng này khởi tạo kết nối với dịch vụ Pinecone bằng khóa API bảo mật. Khóa này thường được lấy từ trình quản lý bí mật (như Google Colab Secrets) để đảm bảo an toàn,.
*   **`INDEX_NAME = 'genai-mas-mcp-ch3'`:** Thiết lập tên cho chỉ mục sẽ được sử dụng xuyên suốt chương.
*   **`NAMESPACE_KNOWLEDGE` và `NAMESPACE_CONTEXT`:** Định nghĩa tên cho hai không gian tên riêng biệt để thực hiện chiến lược RAG kép, cho phép tách biệt tri thức thực tế và hướng dẫn phong cách,.
*   **`spec = ServerlessSpec(cloud='aws', region='us-east-1')`:** Xác định cấu hình hạ tầng **serverless (không máy chủ)** của Pinecone. Đây là kiến trúc được khuyến khích cho các gói miễn phí, chỉ định việc chạy trên dịch vụ đám mây AWS tại khu vực `us-east-1`.

---

**Ví dụ ẩn dụ:**
Hãy tưởng tượng Pinecone là một **thư viện lớn (Index)**. Trong thư viện này, bạn chia làm hai khu vực riêng biệt (**Namespace**): một khu vực chứa **sách bách khoa toàn thư** (KnowledgeStore - nơi chứa sự thật) và một khu vực chứa **sách hướng dẫn kỹ năng viết** (ContextLibrary - nơi chứa cách trình bày). Việc khởi tạo client giống như việc bạn **đăng ký thẻ quản lý thư viện** để có quyền truy cập và sắp xếp các kệ sách này theo đúng vị trí của chúng.

In [8]:
# 2.Initialize Clients
# --- Initialize Clients (assuming this is already done) ---

# --- Initialize Pinecone Client ---
pc = Pinecone(api_key=PINECONE_API_KEY)

# --- Define Index and Namespaces (assuming this is already done) ---
INDEX_NAME = 'genai-mas-mcp-ch3'
NAMESPACE_KNOWLEDGE = "KnowledgeStore"
NAMESPACE_CONTEXT = "ContextLibrary"
spec = ServerlessSpec(cloud='aws', region='us-east-1')

# Check if index exists
if INDEX_NAME not in pc.list_indexes().names():
    print(f"Index '{INDEX_NAME}' not found. Creating new serverless index...")
    pc.create_index(
        name=INDEX_NAME,
        dimension=EMBEDDING_DIM,
        metric='cosine',
        spec=spec
    )
    # Wait for index to be ready
    while not pc.describe_index(INDEX_NAME).status['ready']:
        print("Waiting for index to be ready...")
        time.sleep(1)
    print("Index created successfully. It is new and empty.")
else:
    print(f"Index '{INDEX_NAME}' already exists. Clearing namespaces for a fresh start...")
    index = pc.Index(INDEX_NAME)
    namespaces_to_clear = [NAMESPACE_KNOWLEDGE, NAMESPACE_CONTEXT]

    for namespace in namespaces_to_clear:
        # Check if namespace exists and has vectors before deleting
        stats = index.describe_index_stats()
        if namespace in stats.namespaces and stats.namespaces[namespace].vector_count > 0:
            print(f"Clearing namespace '{namespace}'...")
            index.delete(delete_all=True, namespace=namespace)

            # **CRITICAL FUNCTTION: Wait for deletion to complete**
            while True:
                stats = index.describe_index_stats()
                if namespace not in stats.namespaces or stats.namespaces[namespace].vector_count == 0:
                    print(f"Namespace '{namespace}' cleared successfully.")
                    break
                print(f"Waiting for namespace '{namespace}' to clear...")
                time.sleep(5) # Poll every 5 seconds
        else:
            print(f"Namespace '{namespace}' is already empty or does not exist. Skipping.")

# Connect to the index for subsequent operations
index = pc.Index(INDEX_NAME)


Index 'genai-mas-mcp-ch3' not found. Creating new serverless index...
Index created successfully. It is new and empty.


# 3.Data Preparation: The Context Library (Procedural RAG)

In [9]:
context_blueprints = [
    {
        "id": "blueprint_suspense_narrative",
        "description": "Một Bản thiết kế Ngữ nghĩa (Semantic Blueprint) chính xác được thiết kế để tạo ra các câu chuyện kịch tính và căng thẳng, phù hợp với truyện thiếu nhi. Tập trung vào bầu không khí, các mối đe dọa tiềm tàng và tác động cảm xúc. Lý tưởng cho viết lách sáng tạo.",
        "blueprint": json.dumps({
              "scene_goal": "Tăng cường sự căng thẳng và tạo ra sự hồi hộp.",
              "style_guide": "Sử dụng các câu ngắn, sắc bén. Tập trung vào các chi tiết cảm giác (âm thanh, bóng đổ). Duy trì tông giọng hơi kỳ bí nhưng phù hợp với lứa tuổi.",
              "participants": [
                { "role": "Agent", "description": "Nhân vật chính đang trải qua các sự kiện." },
                { "role": "Source_of_Threat", "description": "Mối nguy hiểm hoặc bí ẩn tiềm ẩn." }
              ],
            "instruction": "Viết lại các dữ kiện đã cung cấp thành một câu chuyện, tuân thủ nghiêm ngặt scene_goal và style_guide."
            })
    },
    {
        "id": "blueprint_technical_explanation",
        "description": "Một Bản thiết kế Ngữ nghĩa được thiết kế để giải thích hoặc phân tích kỹ thuật. Bản thiết kế này tập trung vào sự rõ ràng, tính khách quan và cấu trúc. Lý tưởng để phân tích các quy trình phức tạp, giải thích cơ chế hoặc tóm tắt các phát hiện khoa học.",
        "blueprint": json.dumps({
              "scene_goal": "Giải thích cơ chế hoặc các phát hiện một cách rõ ràng và súc tích.",
              "style_guide": "Duy trì tông giọng khách quan và trang trọng. Sử dụng thuật ngữ chính xác. Ưu tiên độ chính xác thực tế và sự rõ ràng hơn là lối viết kể chuyện hoa mỹ.",
              "structure": ["Định nghĩa", "Chức năng/Vận hành", "Các phát hiện chính/Tác động"],
              "instruction": "Tổ chức các dữ kiện đã cung cấp vào cấu trúc đã định nghĩa, tuân thủ style_guide."
            })
    },
    {
        "id": "blueprint_casual_summary",
        "description": "Một bối cảnh hướng mục tiêu để tạo ra bản tóm tắt thân mật, dễ đọc. Tập trung vào sự ngắn gọn và dễ tiếp cận, giải thích các khái niệm một cách đơn giản.",
        "blueprint": json.dumps({
              "scene_goal": "Tóm tắt thông tin một cách nhanh chóng và thân mật.",
              "style_guide": "Sử dụng ngôn ngữ không trang trọng. Giữ nội dung ngắn gọn và lôi cuốn. Hãy tưởng tượng như đang giải thích cho một người bạn.",
              "instruction": "Tóm tắt các dữ kiện đã cung cấp bằng cách sử dụng hướng dẫn phong cách thân mật (casual style guide)."
            })
    }
]

print(f"\nPrepared {len(context_blueprints)} context blueprints.")


Prepared 3 context blueprints.


In [10]:
knowledge_data_raw = """
Khám phá không gian là việc sử dụng thiên văn học và công nghệ vũ trụ để thám hiểm không gian bên ngoài. Giai đoạn đầu của kỷ nguyên khám phá không gian được thúc đẩy bởi "Cuộc chạy đua vào không gian" giữa Liên bang Xô viết và Hoa Kỳ. Việc Liên Xô phóng vệ tinh Sputnik 1 vào năm 1957 và chuyến đổ bộ lên Mặt Trăng đầu tiên của sứ mệnh Apollo 11 của Mỹ vào năm 1969 là những cột mốc quan trọng.

Chương trình Apollo là chương trình bay vào không gian có người lái của Hoa Kỳ do NASA thực hiện, đã thành công trong việc đưa những con người đầu tiên đổ bộ lên Mặt Trăng. Apollo 11 là sứ mệnh đầu tiên hạ cánh xuống Mặt Trăng, được chỉ huy bởi Neil Armstrong và phi công mô-đun Mặt Trăng Buzz Aldrin, cùng với Michael Collins là phi công mô-đun điều khiển. Bước chân đầu tiên của Armstrong lên bề mặt Mặt Trăng diễn ra vào ngày 20 tháng 7 năm 1969 và được truyền hình trực tiếp trên toàn thế giới. Việc hạ cánh đã yêu cầu Armstrong phải điều khiển thủ công Mô-đun Mặt Trăng Eagle do các thách thức về định vị và tình trạng nhiên liệu thấp.

Juno là một tàu thăm dò không gian của NASA đang bay quanh quỹ đạo Sao Mộc. Nó được phóng vào ngày 5 tháng 8 năm 2011 và tiến vào quỹ đạo cực của Sao Mộc vào ngày 5 tháng 7 năm 2016. Sứ mệnh của Juno là đo đạc thành phần, trọng trường, từ trường và từ quyển vùng cực của Sao Mộc để hiểu rõ cách hành tinh này hình thành. Juno là phi thuyền thứ hai bay quanh quỹ đạo Sao Mộc, sau tàu thăm dò Galileo. Nó được cung cấp năng lượng độc đáo bởi các tấm pin năng lượng Mặt Trời lớn thay vì máy phát nhiệt điện đồng vị phóng xạ (RTGs), giúp nó trở thành sứ mệnh sử dụng năng lượng Mặt Trời xa nhất.

Xe tự hành Sao Hỏa (Mars rover) là một phương tiện cơ giới điều khiển từ xa được thiết kế để di chuyển trên bề mặt Sao Hỏa. NASA JPL đã quản lý thành công một số xe tự hành bao gồm: Sojourner, Spirit, Opportunity, Curiosity và Perseverance. Việc tìm kiếm bằng chứng về khả năng duy trì sự sống và carbon hữu cơ trên Sao Hỏa hiện là mục tiêu hàng đầu của NASA. Perseverance cũng mang theo trực thăng Ingenuity.
"""

In [11]:
#@title 5.Helper Functions for Chunking and Embedding
# -------------------------------------------------------------------------

# Initialize tokenizer for robust, token-aware chunking
tokenizer = tiktoken.get_encoding("cl100k_base")

def chunk_text(text, chunk_size=400, overlap=50):
    """Chunks text based on token count with overlap (Best practice for RAG)."""
    tokens = tokenizer.encode(text)
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk_tokens = tokens[i:i + chunk_size]
        chunk_text = tokenizer.decode(chunk_tokens)
        # Basic cleanup
        chunk_text = chunk_text.replace("\n", " ").strip()
        if chunk_text:
            chunks.append(chunk_text)
    return chunks

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def get_embeddings_batch(texts, model=EMBEDDING_MODEL):
    """Generates embeddings for a batch of texts using OpenAI, with retries."""
    # OpenAI expects the input texts to have newlines replaced by spaces
    texts = [t.replace("\n", " ") for t in texts]
    response = client.embeddings.create(input=texts, model=model)
    return [item.embedding for item in response.data]

In [12]:
#@title 6.Process and Upload Data
# -------------------------------------------------------------------------

# --- 6.1. Context Library ---
print(f"\nProcessing and uploading Context Library to namespace: {NAMESPACE_CONTEXT}")

vectors_context = []
for item in tqdm(context_blueprints):
    # We embed the DESCRIPTION (the intent)
    embedding = get_embeddings_batch([item['description']])[0]
    vectors_context.append({
        "id": item['id'],
        "values": embedding,
        "metadata": {
            "description": item['description'],
            # The blueprint itself (JSON string) is stored as metadata
            "blueprint_json": item['blueprint']
        }
    })

# Upsert data
if vectors_context:
    index.upsert(vectors=vectors_context, namespace=NAMESPACE_CONTEXT)
    print(f"Successfully uploaded {len(vectors_context)} context vectors.")

# --- 6.2. Knowledge Base ---
print(f"\nProcessing and uploading Knowledge Base to namespace: {NAMESPACE_KNOWLEDGE}")

# Chunk the knowledge data
knowledge_chunks = chunk_text(knowledge_data_raw)
print(f"Created {len(knowledge_chunks)} knowledge chunks.")

vectors_knowledge = []
batch_size = 100 # Process in batches

for i in tqdm(range(0, len(knowledge_chunks), batch_size)):
    batch_texts = knowledge_chunks[i:i+batch_size]
    batch_embeddings = get_embeddings_batch(batch_texts)

    batch_vectors = []
    for j, embedding in enumerate(batch_embeddings):
        chunk_id = f"knowledge_chunk_{i+j}"
        batch_vectors.append({
            "id": chunk_id,
            "values": embedding,
            "metadata": {
                "text": batch_texts[j]
            }
        })
    # Upsert the batch
    index.upsert(vectors=batch_vectors, namespace=NAMESPACE_KNOWLEDGE)

print(f"Successfully uploaded {len(knowledge_chunks)} knowledge vectors.")


Processing and uploading Context Library to namespace: ContextLibrary


100%|██████████| 3/3 [00:04<00:00,  1.64s/it]


Successfully uploaded 3 context vectors.

Processing and uploading Knowledge Base to namespace: KnowledgeStore
Created 3 knowledge chunks.


100%|██████████| 1/1 [00:01<00:00,  1.97s/it]

Successfully uploaded 3 knowledge vectors.


In [13]:
#@title 7.Final Verification
# -------------------------------------------------------------------------
print("\nIngestion complete. Final Pinecone Index Stats (may take a moment to update):")
time.sleep(15) # Give Pinecone a moment to update stats
print(index.describe_index_stats())


Ingestion complete. Final Pinecone Index Stats (may take a moment to update):
{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '220',
                                    'content-type': 'application/json',
                                    'date': 'Mon, 29 Dec 2025 01:59:52 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '3',
                                    'x-pinecone-request-id': '637374064766665440',
                                    'x-pinecone-request-latency-ms': '3',
                                    'x-pinecone-response-duration-ms': '4'}},
 'dimension': 1536,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'ContextLibrary': {'vector_count': 3},
                'KnowledgeStore': {'vector_count': 3}},
 'storageFullness': 0.0,
 '